# 03 - Model Selection

This notebook serves to attempt to tune multiple regression models for use in combination 
with pykrige RegressionKriging(). To this end it contains a custom k-fold grid search based
tuning class. Subsequent section contain training/tuning for several different models. 
The final section provides an overview of model performance and optimal parameters found.

## Setup
The first code cell can be edited to define file locations and adjust model in-/outputs. 

In [1]:
data_nodes_input_path = '..\\output\\final_data_nodes.GEOJSON'

x_cols = ['maxspeed', 'bridge', 'junction', 'building_height',
          'dist_to_train', 'dist_to_recreation', 'landuse_is_residential', 
          'landuse_is_commercial', 'landuse_is_industrial', 'rt_highway', 
          'rt_trunk', 'rt_primary', 'rt_secondary','rt_tertiary', 
          'rt_unclassified', 'rt_residential', 'rt_living_street',
          'rt_busway', 'rt_service',]
y_col = 'dBA_reg_adj'
c_cols = ['x', 'y']
random_seed = 12

In [2]:
# default libs
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# vector geo data libs
import pandas as pd
import geopandas as gpd
import contextily as cx
pd.options.mode.copy_on_write = True
pd.set_option('display.max_columns', 500)
# kriging
from pykrige.rk import RegressionKriging
# sklearn
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score
from itertools import product
import re
import random

from tqdm.notebook import tqdm
import os, sys

In [3]:
# Importing data
gdf = gpd.read_file(data_nodes_input_path, engine='pyogrio')
# Retain only data relevant to training/testing
gdf = gdf[gdf[y_col].notna()]

# Utilities for retrieving x/y/coordinate arrays
def get_x_arr(df, cols=x_cols):
    return df[cols].values
def get_y_arr(df, col=y_col):
    return df[col].values
def get_c_arr(df, cols=c_cols):
    return df[cols].values

## PyKrige Tuning Setup
This section contains the custom tuner class (and dependencies). The second cell defines utilities creating spatial folds.


In [4]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

class pykrige_tuning_results(): #todo update to accept list of tuning_score objs
    def __init__(self, score_overview, model):
        # score_overview = dict(sorted(score_overview.items(), key=lambda s: s[1]))
        self.score_overview = score_overview
        self.best_iteration = sorted(score_overview, key=(lambda o: o.score))[0]
        # self.best_score = best_iteration.score        
        # self.best_model_params = list(score_overview.keys())[0][0]
        # self.best_model_params = self._recover_param_dict(self.best_model_params)
        # self.best_krige_params = list(score_overview.keys())[0][1]
        # self.best_krige_params = self._recover_param_dict(self.best_krige_params)

        self.model = model

    def get_optimal_model(self):
        m = self.model(**self.best_iteration.model_params)
        rk_m = RegressionKriging(m, **self.best_iteration.krige_params)
        return rk_m
        
class tuning_score():
    def __init__(self, model_params, krige_params, score):
        self.model_params = model_params
        self.krige_params = krige_params
        self.score = score

class pykrige_rk_tuner():
    """
    Provides an adapted kfold-tuning routine for use with pykrige. Makes several
    assumptions likely specific to this application.
    """
    def __init__(self, data, x_cols, y_col, c_cols=['x', 'y'], fold_col='fold', scorer=root_mean_squared_error):
        self.data = data
        self.x_cols = x_cols
        self.y_col = y_col
        self.c_cols = c_cols
        self.scorer = scorer
        self.fold_col = fold_col

        self.fold_values = self.data[self.fold_col].unique()

    def _create_fold_arr(self, sorted_df, no_folds):
        fold_arr = np.concatenate([np.full((sorted_df.shape[0] // no_folds), i) for i in range(no_folds-1)])
        fold_arr = np.concatenate((fold_arr, np.full(sorted_df.shape[0]-len(fold_arr), no_folds-1)))
        return fold_arr

    def _create_folds(self, no_of_folds=16):
        # no_of_folds must be the square of some integer (i.e. 1,4,9,16,...)
        self.data = self.sort_values(by=['x', 'y']).reset_index(drop=True)
        self.data['xf'] = self._create_fold_arr(self.data, np.sqrt(no_of_folds))
        self.data = self.sort_values(by=['xf', 'y']).reset_index(drop=True)
        self.data['fold'] = self._create_fold_arr(self.data, no_of_folds)
        del self.data['xf']

    def _create_param_dicts_from_grid(self, grid):
        keys = grid.keys()
        values = grid.values()
        combinations = list(product(*values))
        return [dict(zip(keys, combination)) for combination in combinations]
    
    def set_scorer(self, scorer):
        self.scorer = scorer

    def tune(self, model, model_param_grid, krige_param_grid, verbose=False):
        """Custom tuning function for pykrige
        
        args:
        ----------
        model: 
            ML model from sklearn (as per pykrige docs)
        model_param_grid: 
            A dict with (k,v)=(parameter_name:[param_values]), with parameters for the sklearn model. 
            Each possible combination is tested.
        krige_param_grid: 
            A dict with (k,v)=(parameter_name:[param_values]), with parameters for the regression kriging model. 
            Each possible combination is tested.
        """
        model_param_dicts = self._create_param_dicts_from_grid(model_param_grid) 
        krige_param_dicts = self._create_param_dicts_from_grid(krige_param_grid)
        score_tracker = []

        tq = tqdm(total=len(model_param_dicts)*len(krige_param_dicts))
        for m_params in model_param_dicts:
            for k_params in krige_param_dicts:
                if verbose: print(m_params, k_params)
                scores = []
                for fold in self.fold_values: 
                    X_train = self.data[self.data[self.fold_col] != fold][self.x_cols].values
                    c_train = self.data[self.data[self.fold_col] != fold][self.c_cols].values
                    y_train = self.data[self.data[self.fold_col] != fold][self.y_col].values

                    X_test = self.data[self.data[self.fold_col] == fold][self.x_cols].values
                    c_test = self.data[self.data[self.fold_col] == fold][self.c_cols].values
                    y_test = self.data[self.data[self.fold_col] == fold][self.y_col].values
                    
                    m = model(**m_params)
                    m_rk = RegressionKriging(regression_model=m, verbose=False, **k_params)
                    with HiddenPrints():
                        m_rk.fit(X_train, c_train, y_train)
                    y_pred = m_rk.predict(X_test, c_test)
                    scores.append(self.scorer(y_pred, y_test))
                
                score_tracker.append(tuning_score(m_params, k_params, np.mean(scores)))
                    #[(str(m_params.items()), str(k_params.items()))] = np.mean(scores)
                tq.update()
                    
        return pykrige_tuning_results(score_tracker, model)
        # score_overview = dict(sorted(score_overview.items(), key=lambda s: s[1]))
        # self.score_overview = score_overview
        # self.best_score = list(score_overview.values())[0]
        # self.best_model_params = list(score_overview.keys())[0][0]
        # self.best_krige_params = list(score_overview.keys())[0][1]
    
    # def train_model(self, model):
    #     if not self.best_model_params: Throw
    #     if not self.best_krige_params: Throw

        


In [5]:
# spatial fold utils


def create_spatial_folds(df, no_of_folds=16):

    def create_fold_arr(sorted_df, no_folds):
        fold_arr = np.concatenate([np.full((sorted_df.shape[0] // no_folds), i) for i in range(no_folds-1)])
        fold_arr = np.concatenate((fold_arr, np.full(sorted_df.shape[0]-len(fold_arr), no_folds-1)))
        return fold_arr
    # no_of_folds must be the square of some integer (i.e. 1,4,9,16,...)
    df = df.sort_values(by=['x', 'y']).reset_index(drop=True)
    df['xf'] = create_fold_arr(df, int(np.sqrt(no_of_folds)))
    df = df.sort_values(by=['xf', 'y']).reset_index(drop=True)
    return create_fold_arr(df, no_of_folds)

def spatial_train_test_split(gdf, no_test_folds, random_state=1, fold_col='fold'):
    """
    spatial folds
    """
    random.seed(random_state)
    folds = gdf[fold_col].unique()
    random.shuffle(folds)
    return gdf[gdf[fold_col].isin(folds[:no_test_folds])], gdf[gdf[fold_col].isin(folds[no_test_folds:])] 

## Model Tuning

In [6]:
# * setup
score_overview_df = pd.DataFrame(data={'model': [],
                                      'RMSE base': [],
                                      'R2 base': [],
                                      'RMSE tuned': [],
                                      'R2 tuned': [],})

# utility to get the scores
def get_scores_for_m(m, train, test):
    if isinstance(m, RegressionKriging):
        km = m
    else:
        try: 
            km = RegressionKriging(m)
        except:
            raise ValueError('get_scores_for_m() function must be provided with either a regression or a regressionkriging model.')
    
    km.fit(get_x_arr(train), get_c_arr(train), get_y_arr(train))
    y_pred = km.predict(get_x_arr(test),
                        get_c_arr(test))
    r2 = r2_score(get_y_arr(test), y_pred)
    rmse = root_mean_squared_error(get_y_arr(test), y_pred)
    return r2, rmse


# Defining spatial folds
gdf['fold'] = create_spatial_folds(gdf, 16)
gdf_train, gdf_test = spatial_train_test_split(gdf, 4, random_state=random_seed)



# Regression kriging parameters to tune
rk_param_grid = {
    'n_closest_points': [20, 30] ,
    'nlags': [3, 6, 9] # unimportant in testing
}

x_cols2 = ['maxspeed', 'bridge', 'junction', 'building_height',
          'dist_to_train', 'dist_to_recreation']#, 'landuse_is_residential', 
        #   'landuse_is_commercial', 'landuse_is_industrial']
# Regression kriging tuner
rk_tuner = pykrige_rk_tuner(data=gdf_train,
                            x_cols=x_cols2,
                            y_col=y_col)


In [7]:
print(sorted([1,3,2,5]))

[1, 2, 3, 5]


### Linear Regression - ols

In [23]:
LinearRegression().__class__.__name__

'LinearRegression'

In [8]:
# pre tuning scores
m = LinearRegression
r2_base, rmse_base = get_scores_for_m(m(), gdf_train, gdf_test)

# tuning
ols_param_grid = {}
ols_tune_results = rk_tuner.tune(m, ols_param_grid, rk_param_grid)

# post tuning scores
r2_tuned, rmse_tuned = get_scores_for_m(ols_tune_results.get_optimal_model(),
                                        gdf_train, gdf_test)

# scoring evaluations, discarding unnecessary vars
score_overview_df.loc[score_overview_df.shape[0]] = \
    [m().__class__.__name__, rmse_base, r2_base, rmse_tuned, r2_tuned]
del m

Finished learning regression model
Finished kriging residuals


  0%|          | 0/6 [00:00<?, ?it/s]

Finished learning regression model
Finished kriging residuals


### Random Forest - rf

In [9]:
# pre tuning scores
m = RandomForestRegressor
r2_base, rmse_base = get_scores_for_m(m(random_state=random_seed), gdf_train, gdf_test)

# tuning
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [2,4,6,8,10],
    'random_state':[random_seed]
}
rf_tune_results = rk_tuner.tune(m, rf_param_grid, rk_param_grid)

# post tuning scores
r2_tuned, rmse_tuned = get_scores_for_m(rf_tune_results.get_optimal_model(),
                                        gdf_train, gdf_test)

# scoring evaluations, discarding unnecessary vars
score_overview_df.loc[score_overview_df.shape[0]] = \
    [m().__class__.__name__, rmse_base, r2_base, rmse_tuned, r2_tuned]
del m

Finished learning regression model
Finished kriging residuals


  0%|          | 0/3240 [00:00<?, ?it/s]

Finished learning regression model
Finished kriging residuals


### Gradient Boosting - gb

In [10]:
# pre tuning scores
m = GradientBoostingRegressor
r2_base, rmse_base = get_scores_for_m(m(random_state=random_seed), gdf_train, gdf_test)

# tuning
gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [2,4,6,8,10],
    'random_state':[random_seed]
}
gb_tune_results = rk_tuner.tune(m, gb_param_grid, rk_param_grid)

# post tuning scores
r2_tuned, rmse_tuned = get_scores_for_m(gb_tune_results.get_optimal_model(),
                                        gdf_train, gdf_test)

# scoring evaluations, discarding unnecessary vars
score_overview_df.loc[score_overview_df.shape[0]] = \
    [m().__class__.__name__, rmse_base, r2_base, rmse_tuned, r2_tuned]
del m

Finished learning regression model
Finished kriging residuals


  0%|          | 0/9720 [00:00<?, ?it/s]

Finished learning regression model
Finished kriging residuals


### GaussianProcessRegressor - gpr

In [12]:
# pre tuning scores
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct
kernel = DotProduct() + WhiteKernel()
m = GaussianProcessRegressor
r2_base, rmse_base = get_scores_for_m(m(random_state=random_seed), gdf_train, gdf_test)
print('pre-tuned eval done')
import time
time.sleep(3)
# tuning
param_grid = {
    'kernel': [
        RBF(length_scale=1.0),
        RBF(length_scale=0.1),
        Matern(length_scale=1.0, nu=1.5),
        Matern(length_scale=0.1, nu=1.5),
        RationalQuadratic(length_scale=1.0, alpha=1.0),
        RationalQuadratic(length_scale=0.1, alpha=1.0),
        WhiteKernel()
    ],
    'alpha': [1e-7, 1e-4, 1e-2, 1e-1],
    'n_restarts_optimizer': [0, 1, 5]
}
sgd_tune_results = rk_tuner.tune(m, param_grid, rk_param_grid)

# post tuning scores
r2_tuned, rmse_tuned = get_scores_for_m(sgd_tune_results.get_optimal_model(),
                                        gdf_train, gdf_test)

# # scoring evaluations, discarding unnecessary vars
score_overview_df.loc[score_overview_df.shape[0]] = \
    [m().__class__.__name__, rmse_base, r2_base, rmse_tuned, r2_tuned]
del m

Finished learning regression model
Finished kriging residuals
pre-tuned eval done


c:\Users\flori\!projects\noise-these\noise-thesis\.venv\Lib\site-packages\pykrige\ok.py:753: LinAlgWarning: Ill-conditioned matrix (rcond=2.61538e-19): result may not be accurate.
  x = scipy.linalg.solve(a, b)
c:\Users\flori\!projects\noise-these\noise-thesis\.venv\Lib\site-packages\pykrige\ok.py:753: LinAlgWarning: Ill-conditioned matrix (rcond=2.61539e-19): result may not be accurate.
  x = scipy.linalg.solve(a, b)
c:\Users\flori\!projects\noise-these\noise-thesis\.venv\Lib\site-packages\pykrige\ok.py:753: LinAlgWarning: Ill-conditioned matrix (rcond=1.74359e-19): result may not be accurate.
  x = scipy.linalg.solve(a, b)
c:\Users\flori\!projects\noise-these\noise-thesis\.venv\Lib\site-packages\pykrige\ok.py:753: LinAlgWarning: Ill-conditioned matrix (rcond=1.96154e-19): result may not be accurate.
  x = scipy.linalg.solve(a, b)
c:\Users\flori\!projects\noise-these\noise-thesis\.venv\Lib\site-packages\pykrige\ok.py:753: LinAlgWarning: Ill-conditioned matrix (rcond=1.10651e-18): resu

  0%|          | 0/504 [00:00<?, ?it/s]

c:\Users\flori\!projects\noise-these\noise-thesis\.venv\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\flori\!projects\noise-these\noise-thesis\.venv\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\flori\!projects\noise-these\noise-thesis\.venv\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterati

Finished learning regression model
Finished kriging residuals


In [14]:
r2_tuned, rmse_tuned = get_scores_for_m(sgd_tune_results.get_optimal_model(),
                                        gdf_train, gdf_test)

Finished learning regression model
Finished kriging residuals


## Tuning Results

In [ ]:
# Optional pickling function to save tuning results:
"""import pickle
with open('ols_dump', 'wb') as f:
    pickle.dump(ols_tune_results, f)"""

In [26]:
# Score overview graph
score_overview_df.to_clipboard()
score_overview_df

,model,RMSE base,R2 base,RMSE tuned,R2 tuned
0,ABCMeta,6.572276,-0.794307,6.532011,-0.772389
1,ABCMeta,5.065892,-0.066049,4.715172,0.076450
2,ABCMeta,5.135535,-0.095561,4.777596,0.051835
3,type,67.658090,-189.153625,4.953920,-0.019443


In [30]:
# Param overview graph
models = []
param_names = []
param_vals = []
param_opts = []
for name, params, results in [('ols', ols_param_grid, ols_tune_results), ('rf', rf_param_grid, rf_tune_results), ('gdb', gb_param_grid, gb_tune_results), ('gpr', param_grid, sgd_tune_results)]:
    for i, (param, vals) in enumerate(params.items()):
        if param == 'random_state': continue
        if i == 0: models.append(name)
        else: models.append('')

        param_names.append(param)
        param_vals.append(vals)
        param_opts.append(results.best_iteration.model_params[param])

pd.DataFrame(data={'model': models, 'paramater': param_names, 'value range': param_vals, 'optimal value': param_opts})

,model,paramater,value range,optimal value
0,rf,n_estimators,"[50, 100, 200]",50
1,,max_depth,"[None, 10, 20, 30]",10
2,,min_samples_split,"[2, 5, 10]",10
3,,min_samples_leaf,"[1, 2, 4]",4
4,,max_features,"[2, 4, 6, 8, 10]",2
5,gdb,n_estimators,"[50, 100, 200]",50
6,,learning_rate,"[0.01, 0.1, 0.2, 0.3]",0.01
7,,max_depth,"[3, 5, 7]",7
8,,min_samples_split,"[2, 5, 10]",10
9,,min_samples_leaf,"[1, 2, 4]",2
